In [119]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import joblib

# Carregar o dataset
df = pd.read_excel('ready_for_filtering.xlsx')
df.fillna('', inplace=True)

X = df['combined']
y = df['sanitized_metrics']  # Target: métricas a serem recomendadas

# Ajustar parâmetros da vetorização TF-IDF
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),         # Capturar unigramas e bigramas
    max_features=2000,          # Limitar o número de features
)

# Vetorizar os dados textuais com TF-IDF
X_tfidf = vectorizer.fit_transform(X)

# Usar SVD para redução de dimensionalidade
svd = TruncatedSVD(n_components=100, random_state=42)  # Reduzir para 100 dimensões latentes
X_svd = svd.fit_transform(X_tfidf)

# Calcular a similaridade dos cossenos na nova representação (SVD)
cosine_sim_matrix = cosine_similarity(X_svd, X_svd)

# Função para recomendar métricas com base nos perfis mais semelhantes
def recommend_metrics(profile_index, cosine_sim_matrix, df, top_n=5):
    # Mostrar os dados do perfil selecionado (profile_index)
    print(f"Dados do perfil {profile_index}:")
    print(df.iloc[profile_index])
    
    # Similaridade do perfil com todos os outros
    sim_scores = list(enumerate(cosine_sim_matrix[profile_index]))
    
    # Ordenar por similaridade (maior -> menor)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Excluir o próprio perfil (primeira posição) e selecionar os top_n perfis mais semelhantes
    sim_scores = sim_scores[1:top_n+1]
    
    # Mostrar a similaridade dos perfis recomendados
    print(f"\nPerfis mais semelhantes ao perfil {profile_index}:")
    for i, score in sim_scores:
        print(f"Perfil {i}: Similaridade = {score*100:.2f}% - Métrica recomendada: {df['sanitized_metrics'].iloc[i]}")
    
    # Recomendar as métricas dos perfis mais semelhantes
    recommended_metrics = [df['sanitized_metrics'].iloc[i] for i, _ in sim_scores]
    
    return recommended_metrics

# Exemplo: recomendar métricas para o perfil com índice 432
profile_index = 432
recommended_metrics = recommend_metrics(profile_index, cosine_sim_matrix, df)

# Exibir métricas recomendadas
print("\nMétricas recomendadas:", recommended_metrics)

import joblib

# Salvar o vetor TF-IDF para reutilização posterior
joblib.dump(vectorizer, 'tfidf_vectorizer_for_recommendation.joblib')

# Salvar o modelo SVD para reutilização posterior
joblib.dump(svd, 'svd_model_for_recommendation.joblib')

# Salvar o dataset SVD-transformado para futuras comparações
X_svd_transformed = svd.transform(X_tfidf)  # Transform the entire dataset
joblib.dump(X_svd_transformed, 'svd_transformed_data_for_recommendation.joblib')

print("Modelos e dados transformados SVD salvos com sucesso!")

Dados do perfil 432:
Carimbo de data/hora                                             09:54:14
id_integer                                                            176
role                                                          Team leader
years_exp                                                         10 a 20
org_size                                                    Média empresa
agile_methods                     ['Scrum', 'ScrumBan', 'Kanban', 'Lean']
use_metrics_planning                                                  Não
use_metrics_review                                                    Sim
use_metrics_weekly                                                    Sim
use_metrics_daily                                                     Não
use_metrics_retro                                                     Sim
metrics_category                      Pessoas, Produto, Processo, Cliente
sanitized_metrics                                               Lead time
combined         